# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [23]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
print(cassandra.__version__)

3.24.0


#### Creating list of filepaths to process original event csv data files

In [24]:
# checking your current working directory
# print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [25]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


643


In [26]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

549


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [29]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [31]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        {'class' : 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [32]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [34]:
create_table = "create table if not exists songs"
create_table = create_table + '(session_id int, item_in_session int, artist text, song_title text, song_length float, primary key(session_id, item_in_session))'

try:
    session.execute(create_table)
except Exception as e:
    print(e)

In [60]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query_1 = 'select artist, song_title, song_length from songs where session_id=%s and item_in_session=%s'
                    

In [35]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into songs(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "values (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [61]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query_1, (818, 34))
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Martina Topley Bird', song_title='Poison', song_length=177.08363342285156)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [44]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_table_2 = 'create table if not exists artists'
create_table_2 = create_table_2 + '(user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_first_name text, user_last_name text, primary key(user_id, session_id, item_in_session))'

try:
    session.execute(create_table_2)
except Exception as e:
    print(e)

In [51]:
query_2 = 'select artist_name, song_title, user_first_name, user_last_name from artists where user_id=%s and session_id=%s'

In [47]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into artists(user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [52]:
try:
    rows = session.execute(query_2, (49, 816))
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist_name='The Rascals', song_title='People Got To Be Free', user_first_name='Chloe', user_last_name='Cuevas')
Row(artist_name='A Day To Remember', song_title='My Life For Hire', user_first_name='Chloe', user_last_name='Cuevas')
Row(artist_name='Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', song_title='Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', user_first_name='Chloe', user_last_name='Cuevas')
Row(artist_name='Train', song_title='Shelter Me', user_first_name='Chloe', user_last_name='Cuevas')
Row(artist_name="Guns N' Roses", song_title='Sympathy For The Devil', user_first_name='Chloe', user_last_name='Cuevas')
Row(artist_name='Kings Of Leon', song_title='Revelry', user_first_name='Chloe', user_last_name='Cuevas')
Row(artist_name="The Pretend It's December Choir", song_title="It's Christmas So We'll Stop (Choir Version)", user_first_name='Chloe', user_last_name='Cuevas')
Row(artist_name='Vampire Weekend', song_title='Giving Up T

In [54]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_table_3 = 'create table if not exists users'
create_table_3 = create_table_3 + '(song_title text, user_id int, user_first_name text, user_last_name text, primary key((song_title), user_id))'

try:
    session.execute(create_table_3)
except Exception as e:
    print(e)
                    

In [55]:
query_3 = 'select user_first_name, user_last_name from users where song_title=%s'

In [57]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into users(song_title, user_id, user_first_name, user_last_name)"
        query = query + "values (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [59]:
try:
    rows = session.execute(query_3, ('All Hands Against His Own',))
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(user_first_name='Sara', user_last_name='Johnson')


### Drop the tables before closing out the sessions

In [62]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS songs")
session.execute("DROP TABLE IF EXISTS artists")
session.execute("DROP TABLE IF EXISTS users")

### Close the session and cluster connection¶

In [63]:
session.shutdown()
cluster.shutdown()